# Part 1

In [1]:
class Game:
    
    def __init__(self, p1: int, p2: int):
        self.deterministic_die = 100
        
        # positions
        self.p1 = p1
        self.p2 = p2
        
        self.score1 = 0
        self.score2 = 0
        
        self.num_die_rolls = 0
        
        self.is_over = False
    
    def roll_die(self):
        self.deterministic_die += 1
        if self.deterministic_die == 101:
            self.deterministic_die = 1
        return self.deterministic_die
        
    def play_round(self):
        # Player 1's turn
        steps = sum(self.roll_die() for _ in range(3))
        self.p1 += steps
        self.p1 = self.p1 % 10
        if self.p1 == 0:
            self.p1 = 10
        self.score1 += self.p1
        self.num_die_rolls += 3
        
        if self.score1 >= 1000:
            self.is_over = True
            return
        
        # Player 2's turn
        steps = sum(self.roll_die() for _ in range(3))
        self.p2 += steps
        self.p2 = self.p2 % 10
        if self.p2 == 0:
            self.p2 = 10
        self.score2 += self.p2
        self.num_die_rolls += 3
        
        if self.score2 >= 1000:
            self.is_over = True

In [2]:
def part1(p1, p2):
    game = Game(p1, p2)
    while not game.is_over:
        game.play_round()
    
    losing_score = min(game.score1, game.score2)
    return losing_score * game.num_die_rolls
        

# Test Case
assert part1(4, 8) == 739785

# Get real answer
print("Answer to Part 1:", part1(7, 8))

Answer to Part 1: 556206


# Part 2

In [3]:
from collections import defaultdict
from copy import deepcopy


class MultiverseGame:
    
    # Sum of die outcome -> number of multiverse branches where the outcome occurs
    DIRAC_OUTCOMES = {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}
    
    WINNING_SCORE = 21
    
    def __init__(self, p1: int, p2: int):
        self.multi_positions = defaultdict(lambda: defaultdict(int))
        # This is the root universe.
        # Players start at starting positions with scores of 0.
        self.multi_positions[(p1, p2)][0, 0] += 1
        
        self.player_1_wins = 0
        self.player_2_wins = 0
        
    def get_new_positions(self, curr_pos: int):
        new_positions = defaultdict(int)
        for roll_sum, count in self.DIRAC_OUTCOMES.items():
            new_position = (curr_pos + roll_sum) % 10
            if new_position == 0:
                new_position = 10
            new_positions[new_position] += count
        return new_positions
    
    def play_round(self):
        new_multi_positions = defaultdict(lambda: defaultdict(int))
        
        # Possible positions, and possible scores associated with each possible position
        for (p1, p2), scores_dict in self.multi_positions.items():

            # Player 1's turn to roll Dirac Dice
            new_positions1 = self.get_new_positions(p1)
            # Player's 2 turn to roll Dirac Dice
            new_positions2 = self.get_new_positions(p2)
            
            # Calculate new multiverses' scores
            for new_p1, count1 in new_positions1.items():
                for (score1, score2), score_count in scores_dict.items():
                    # First see how player 1 is doing
                    if score1 > self.WINNING_SCORE or score2 > self.WINNING_SCORE:
                        raise Exception("You made a mistake!)")

                    new_score1 = score1+new_p1
                    if new_score1 >= self.WINNING_SCORE:
                        self.player_1_wins += count1*score_count
                        continue
                    
                    # Now we factor in player 2 into the multiverses
                    for new_p2, count2 in new_positions2.items():
                        new_count = count1*count2*score_count
                        
                        new_score2 = score2+new_p2
                        if new_score2 >= self.WINNING_SCORE:
                            self.player_2_wins += new_count
                            continue
                        
                        new_both_pos = (new_p1, new_p2)
                        new_scores = (new_score1, new_score2)
                        new_multi_positions[new_both_pos][new_scores] += new_count
                

                    

                        
        self.multi_positions = new_multi_positions

In [4]:
def part2(p1, p2):
    game = MultiverseGame(p1, p2)
    i = 0
    while game.multi_positions:
        # There's a multiverse where no one has explored yet,
        # so keep exploring the multiverses.
        game.play_round()
        if i > 21:
            raise Exception("Your loop is going on for too long!")
    
    return game


# Test case
test_game = part2(4, 8)
test_game.player_1_wins == 444356092776315
test_game.player_2_wins == 341960390180808

# Actual case
game = part2(7, 8)
answer = max(game.player_1_wins, game.player_2_wins)
print(f"The player who won in the most universes won games in {answer} universes.")

The player who won in the most universes won games in 630797200227453 universes.
